## `BeautifulSoup`

In [ ]:
import re
import csv
import os
from bs4 import BeautifulSoup

# title: <span class="player-hero__title__content">
# description: <p class="talk-description" lang="en">
# views: <span class="talk-sharing__value"> 96,013 </span>


def parse(soup):
    # both title and views are can be parsed in separate tags.
    title = soup.find('span', {'class' : "player-hero__title__content"}).text.strip('\n')
    views = soup.find('span', {'class' : "talk-sharing__value"}).text.strip('\n')
    descr = soup.find('p', {'class' : "talk-description"}).text.strip('\n')
    return title.strip(), views, descr

def to_csv(pth, out):
    # open file to write to.
    with open(out, "w") as out:
        # create csv.writer.
        wr = csv.writer(out)
        # write our headers.
        wr.writerow(["title", "views", "descr"])
        # get all our html files.
        for html in os.listdir(pth):
            with open(os.path.join(pth, html)) as f:
                print(html)
                # parse the file and write the data to a row.
                wr.writerow(parse(BeautifulSoup(f, "lxml")))

to_csv("./html_files/descriptions/","descriptions-2.csv") # This is to the test directory!

## `pandas`

In [24]:
import pandas
colnames = ['title', 'views' , 'descr']
data = pandas.read_csv('./data/descriptions-2.csv', names=colnames)
titles = data.title.tolist()
views = data.views.tolist()
descriptions = data.descr.tolist()

In [44]:
print(type(descriptions[1]))

<class 'str'>


## `sklearn`

In [ ]:
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import jaccard_similarity_score

tokenize = lambda doc: doc.lower().split()
 
sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
sklearn_representation = sklearn_tfidf.fit_transform(descriptions)
#print(sklearn_representation[0])

In [78]:
# To get a list of punctuation:

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [96]:
# To find out the ordinal of a particular vexing character:

somewords = descriptions[2]
print(ord(somewords(16))

SyntaxError: unexpected EOF while parsing (<ipython-input-96-7d76c83bc369>, line 4)

In [134]:
from stop_words import get_stop_words


def ourtokens(ourstring):
    
    stoplist = set(get_stop_words('en'))
    finalList = []
    
    wordList = ourstring.lower().split()
    for i in range(len(wordList)):
        #wordList[i] = re.sub('[^a-zA-Z\']', '', wordList[i]).strip(chr(8212)) 
        #NOTE: the above left spaces and added empty strings
        
        no_punc = wordList[i].strip(string.punctuation) #remove most punctuation
        no_emphwhatever = no_punc.strip(chr(8212)) # remove that weirdness
        no_num = no_emphwhatever.strip(string.digits) #remove numbers
        
        if (len(no_num) > 0) and (no_num not in stoplist): # Requires stop_words
            # First conditional stops empty strings from being added
            finalList.append(no_num)
            
    return finalList

def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    #print(intersection)
    union = set(query).union(set(document))
    #print(union)
    return len(intersection)/len(union)

jaccard_similarity(ourtokens(descriptions[1]), ourtokens(descriptions[2]))

0.021739130434782608

In [135]:
import csv

des_word_lists = []
for i in range(len(descriptions)):
    # Create list of words for each description
    words = ourtokens(descriptions[i])
    des_word_lists.append({'descriptions': descriptions[i], 'words': words})
    
    # Tells you where you are in the rows
    if (i % 100) == 0:
        print(str(i) + " rows completed")
                           
# Creates the new CSV
with open('desPlusWords.csv', 'w') as desfile:
    fields = ['descriptions', 'words']
    writer = csv.DictWriter(desfile, fieldnames = fields)
    
    writer.writeheader()
    writer.writerows(des_word_lists)
    

0 rows completed
100 rows completed
200 rows completed
300 rows completed
400 rows completed
500 rows completed
600 rows completed
700 rows completed
800 rows completed
900 rows completed
1000 rows completed
1100 rows completed
1200 rows completed
1300 rows completed
1400 rows completed
1500 rows completed
1600 rows completed
1700 rows completed
1800 rows completed
1900 rows completed
2000 rows completed
2100 rows completed
2200 rows completed


In [144]:
import numpy

Ndes = len(des_word_lists)

# From http://stackoverflow.com/questions/568962/how-do-i-create-an-empty-array-matrix-in-numpy
jac_mat = numpy.zeros(shape=(Ndes,Ndes))

for i in range(Ndes):
    if (i % 100) == 0:
        print(str(i) + " rows completed")  
    
    # Start the pairwise computations
    
    for j in range((i+1),Ndes):
        # Pull the ith and jth document
        doc_i = des_word_lists[i]['words']
        doc_j = des_word_lists[j]['words']
        
        # Get the Jaccard similarity
        jac_ij = jaccard_similarity(doc_i, doc_j)
        
        # Since the Jaccard will be the same between i and j as it will between
        # j and i, we set JAC_MAT[i,j] and JAC_MAT[j,i] to be the same value
        jac_mat[i,j] = jac_ij
        jac_mat[j,i] = jac_ij
        


0 rows completed
100 rows completed
200 rows completed
300 rows completed
400 rows completed
500 rows completed
600 rows completed
700 rows completed
800 rows completed
900 rows completed
1000 rows completed
1100 rows completed
1200 rows completed
1300 rows completed
1400 rows completed
1500 rows completed
1600 rows completed
1700 rows completed
1800 rows completed
1900 rows completed
2000 rows completed
2100 rows completed
2200 rows completed


In [143]:
# This block find the maximum for the matrix

# Initialize the max to be zero. 
mat_max = 0

# Loop over all the rows
for i in range(Ndes):
    # Find the maximum for each row
    row_max = max(jac_mat[i])
    
    # Check if the current row's maximum is higher than the current MAT_MAX.
    # If the row maximum is bigger, then set MAT_MAX to the row maximum.
    if row_max > mat_max:
        mat_max = row_max

print(mat_max)

0.304347826087


In [154]:
jac_mean = jac_mat.mean()
print(jac_mean, mat_max)

0.0097816531347 0.304347826087


In [157]:
print("The mean is {}, and the max is {}.".format(jac_mean, mat_max))

The mean is 0.009781653134697772, and the max is 0.30434782608695654.


In [159]:
import networkx as nx
G = nx.from_numpy_matrix(jac_mat)
nx.draw(G)